# 关键概念

欢迎来到``brainscale``的世界！

``brainscale``是一个支持动力学神经网络模型的在线学习Python库。在线学习（online learning）是一种学习范式，它允许模型在不断地接收新数据的同时，不断地更新自己的参数。这种学习方式在许多现实世界的应用中非常有用，比如在机器人控制、智能体的决策制定、以及大规模数据流的处理中。

在这个章节，我将会介绍一些关键概念，这些概念是理解和使用``brainscale``在线学习的基础。这些概念包括：

- 如何构建支持在线学习的high-level神经网络模型。
- 用于定制化网络模块的 模型状态``ETraceState``、模型参数``ETraceParam``和 模型交互算子``ETraceOp``。
- 在线学习算法 ``ETraceAlgorithm``。

``brainscale``精密地整合在以``brainstate``为中心的[脑动力学编程生态系统](https://ecosystem-for-brain-dynamics.readthedocs.io/)中。我们强烈建议您首先熟悉[``brainstate``的基本用法](https://brainstate.readthedocs.io/)，以此能帮助您更好地理解``brainscale``的工作原理。

In [11]:
import brainstate as bst
import brainunit as u

import brainscale

## 1. ``brainscale``支持的动力学模型

``brainscale``并非支持任意动力学模型的在线学习。它目前支持的动力学模型具有如下图所示的结构，即“动力学（dynamics）”和“动力学之间的交互（interaction）”是严格分开的。这种结构的模型可以被分解为两个部分：

- **动力学部分**：这部分模型描述了神经元内在的动力学行为，比如LIF神经元模型、FitzHugh-Nagumo模型、长短期记忆网络（LSTM）等。动力学状态（hidden states）的更新是严格逐元素运算的（element-wise operations），但模型可以包含多个动力学状态。
- **交互部分**：这部分模型描述了神经元之间的交互，比如权重矩阵、连接矩阵等。模型动力学之间的交互可以实现为标准的矩阵乘法、卷积操作、稀疏操作等。


![](../_static/model-dynamics-supported.png)



让我们以一个简单网络模型的例子来说明``brainscale``支持的动力学模型。我们考虑一个简单的LIF神经元网络，它的动力学部分由如下的微分方程描述：

$$
\begin{aligned}
\tau \frac{dv_i}{dt} &= -v_i + I_{\text{ext}} + s_i \\
\tau_s \frac{ds_i}{dt} &= -s_i + \sum_{j} w_{ij} \delta(t - t_j)
\end{aligned}
$$

其中，$v_i$是神经元的膜电位，当神经元膜电位超过阈值$v_{th}$时，神经元会发放动作电位，并且膜电位会被重置为$v_{\text{reset}}$。

$$
\begin{aligned}
z_i & = \mathcal{H}(v_i-v_{th}) \\
v_i & \leftarrow v_{\text{reset}}
\end{aligned}
$$

另外，$s_i$是突触后电流，$I_{\text{ext}}$是外部输入电流，$w_{ij}$是神经元$i$到神经元$j$的突触权重，$\delta(t - t_j)$是Dirac函数，表示在时间$t_j$接收到一个突触事件。$\tau$和$\tau_s$分别是膜电位和突触后电流的时间常数。

通过数值积分的方法，我们离散化上述微分方程，并写成向量的形式，可以得到如下的动力学更新规则：

$$
\begin{aligned}
\mathbf{v}_i^{t+1} &= \mathbf{v}_i^{t} + \frac{\Delta t}{\tau} (-\mathbf{v}_i^{t} + \mathbf{I}_{\text{ext}} + \mathbf{s}^t) \\
\mathbf{s}_i^{t+1} &= \mathbf{s}_i^{t} + \frac{\Delta t}{\tau_s} (-\mathbf{s}_i^{t} + \underbrace{  W \mathbf{z}^t  } _ {\text{neuronal interaction}} )
\end{aligned}
$$

可以看到，LIF神经元的动力学部分的更新是逐元素的，而交互部分的更新是通过矩阵乘法实现的。``brainscale``支持的动力学模型都可以被分解为这样的动力学部分和交互部分。值得注意的是，基本上大部分的循环神经网络模型都满足这样的结构，因此``brainscale``可以支持大部分循环神经网络模型的在线学习。


## 2. ``brainscale.nn``：构建支持在线学习的神经网络

在``brainscale``中，我们可以完全使用与``brainstate``一模一样的语法来构建支持在线学习的神经网络模型。具体教程可以参考 [构建人工神经网络](https://brainstate.readthedocs.io/en/latest/tutorials/artificial_neural_networks-zh.html) 和 [构建脉冲神经网络](https://brainstate.readthedocs.io/en/latest/tutorials/spiking_neural_networks-zh.html)。

但是，唯一不一样的地方在于，我们要使用[``brainscale.nn``模块](../apis/nn.rst)中的组件来构建神经网络模型。这些组件是``brainstate.nn``的扩展，它们是支持在线学习的单元模块。

下面是一个简单的例子，展示了如何使用``brainscale.nn``模块来构建一个简单的LIF神经元网络。

In [12]:
class LIF_Delta_Net(bst.nn.Module):
    def __init__(
        self,
        n_in,
        n_rec,
        tau_mem=5. * u.ms,
        V_th=1. * u.mV,
        spk_fun=bst.surrogate.ReluGrad(),
        spk_reset: str = 'soft',
        rec_scale: float = 1.,
        ff_scale: float = 1.,
    ):
        super().__init__()

        # 使用 brainscale.nn 内的 LIF 模型
        self.neu = brainscale.nn.LIF(n_rec, tau=tau_mem, spk_fun=spk_fun, spk_reset=spk_reset, V_th=V_th)

        # 构建输入和循环连接权重
        rec_init = bst.init.KaimingNormal(rec_scale, unit=u.mV)
        ff_init = bst.init.KaimingNormal(ff_scale, unit=u.mV)
        w_init = u.math.concatenate([ff_init([n_in, n_rec]), rec_init([n_rec, n_rec])], axis=0)

        # 使用 delta 突触投射來构建输入和循环连接
        self.syn = bst.nn.DeltaProj(
            # 使用 brainscale.nn 内的 Linear 模型
            comm=brainscale.nn.Linear(n_in + n_rec, n_rec, w_init=w_init, b_init=bst.init.ZeroInit(unit=u.mV)),
            post=self.neu
        )

    def update(self, spk):
        inp = u.math.concatenate([spk, self.neu.get_spike()], axis=-1)
        self.syn(inp)
        self.neu()
        return self.neu.get_spike()

在这个例子中，我们定义了一个``LIF_Delta_Net``类，它继承自``bst.nn.Module``。这个类包含了一个LIF神经元模型``self.neu``和一个``DeltaProj``连接模块``self.syn``。``DeltaProj``模块用于构建输入和循环连接。

接下来，我们构建一个三层的门控循环单元（GRU）神经网络模型：

In [13]:
class GRU_Net(bst.nn.Module):
    def __init__(
        self,
        n_in: int,
        n_rec: int,
        n_out: int,
        n_layer: int,
    ):
        super().__init__()

        # 构建 GRU 层
        self.layers = []
        for i in range(n_layer - 1):
            # 使用 brainscale.nn 内的 GRUCell 模型
            self.layers.append(brainscale.nn.GRUCell(n_in, n_rec))
            n_in = n_rec
        self.layers.append(brainscale.nn.GRUCell(n_in, n_out))

    def update(self, x):
        # 更新 GRU 层
        for layer in self.layers:
            x = layer(x)
        return x

可以看到，基于[``brainscale.nn``模块](../apis/nn.rst)构建神经网络模型和基于[``brainstate.nn``模块](https://brainstate.readthedocs.io/en/latest/apis/nn.html)构建神经网络模型的过程是一样的。这意味着，您可以直接使用``brainstate``的教程来构建支持在线学习的神经网络模型。

## 3. ``ETraceState``、``ETraceParam``和``ETraceOp``：定制化网络模块

尽管``brainscale.nn``模块提供了一些基本的网络模块，但却无法涵盖所有可能的网络动力学。因此，我们有必要提供一种机制，允许用户定制化网络模块。在``brainscale``中，我们提供了``ETraceState``、``ETraceParam``和``ETraceOp``这三个类，用于定制化网络模块。

- ``brainscale.ETraceState``：对应于模块中的模型状态$\mathbf{h}$。它用于定义模型的动力学状态，比如LIF神经元的膜电位、指数突触模型的突触后电导等。
- ``brainscale.ETraceParam``：对应于模块中的模型参数$\theta$。它用于定义模型的参数，比如线性矩阵乘法的权重矩阵。也可以用于定义LIF神经元中需要自适应学习的时间常数等。凡是需要在模型的训练过程中进行梯度更新的参数，都应该定义在``ETraceParam``中。
- ``brainscale.ETraceOp``：对应于模块中的输入数据如何基于模型参数得到突触后电流的操作。它用于定义模型的操作，比如线性矩阵乘法、稀疏矩阵乘法、卷积操作等。

``ETraceState``、``ETraceParam``和``ETraceOp``是``brainscale``的三个基本概念，它们是构建支持在线学习的神经网络模型的基础。

接下来，让我们以一系列简单的例子来说明如何使用``ETraceState``、``ETraceParam``和``ETraceOp``来定制化网络模块。

### 3.1 ``ETraceState``：模型状态

我们首先考虑一个简单的LIF神经元模型，它的动力学部分由如下的微分方程描述：

$$
\begin{aligned}
\tau \frac{dv_i}{dt} &= -v_i + I_{\text{ext}} + v_\text{rest} \\
z_i & = \mathcal{H}(v_i-v_{th}) \\
v_i & \leftarrow v_{\text{reset}} \quad \text{if} \quad z_i > 0
\end{aligned}
$$

其中，$v_i$是神经元的膜电位，当神经元膜电位超过阈值$v_{th}$时，神经元会发放动作电位，并且膜电位会被重置为$v_{\text{reset}}$。$\mathcal{H}$是一个阶跃函数，表示神经元的发放动作电位，$I_{\text{ext}}$是外部输入电流，$\tau$是膜电位的时间常数，$v_\text{rest}$ 是膜电位的静息电位。

In [14]:
import jax
from typing import Callable


class LIF(bst.nn.Neuron):
    """
    Leaky integrate-and-fire neuron model.
    """

    def __init__(
        self,
        size: bst.typing.Size,
        keep_size: bool = False,
        R: bst.typing.ArrayLike = 1. * u.ohm,
        tau: bst.typing.ArrayLike = 5. * u.ms,
        V_th: bst.typing.ArrayLike = 1. * u.mV,
        V_reset: bst.typing.ArrayLike = 0. * u.mV,
        V_rest: bst.typing.ArrayLike = 0. * u.mV,
        V_initializer: Callable = bst.init.Constant(0. * u.mV),
        spk_fun: Callable = bst.surrogate.ReluGrad(),
        spk_reset: str = 'soft',
        name: str = None,
    ):
        super().__init__(size, keep_size=keep_size, name=name, spk_fun=spk_fun, spk_reset=spk_reset)

        # parameters
        self.R = bst.init.param(R, self.varshape)
        self.tau = bst.init.param(tau, self.varshape)
        self.V_th = bst.init.param(V_th, self.varshape)
        self.V_rest = bst.init.param(V_rest, self.varshape)
        self.V_reset = bst.init.param(V_reset, self.varshape)
        self.V_initializer = V_initializer

    def init_state(self, batch_size: int = None, **kwargs):
        # 这里是最关键的一步，我们定义了一个 ETraceState 类，用于描述膜电位的动力学状态
        self.V = brainscale.ETraceState(bst.init.param(self.V_initializer, self.varshape, batch_size))

    def reset_state(self, batch_size: int = None, **kwargs):
        self.V.value = bst.init.param(self.V_initializer, self.varshape, batch_size)

    def get_spike(self, V=None):
        V = self.V.value if V is None else V
        v_scaled = (V - self.V_th) / (self.V_th - self.V_reset)
        return self.spk_fun(v_scaled)

    def update(self, x=0. * u.mA):
        last_v = self.V.value
        lst_spk = self.get_spike(last_v)
        V_th = self.V_th if self.spk_reset == 'soft' else jax.lax.stop_gradient(last_v)
        V = last_v - (V_th - self.V_reset) * lst_spk
        # membrane potential
        dv = lambda v: (-v + self.V_rest + self.R * self.sum_current_inputs(x, v)) / self.tau
        V = bst.nn.exp_euler_step(dv, V)
        V = self.sum_delta_inputs(V)
        self.V.value = V
        return self.get_spike(V)

在上面的代码中，我们继承``brainstate.nn.Neuron``定义了这个``LIF``模型。这个类包含了一个``ETraceState``类变量``self.V``，用于描述膜电位的动力学状态。在``init_state``方法中，我们初始化了膜电位的动力学状态。在``update``方法中，我们更新了膜电位的动力学状态。实际上，这个类的定义与``brainstate``中的``LIF``类的定义基本上是一模一样的，唯一不同的地方在于``brainstate``使用``brainstate.HiddenState``来描述膜电位的动力学状态，而``brainscale``使用``brainstate.ETraceState``来标记该膜电位的动力学状态是需要用于在线学习的。

因此，我们可以说，``brainscale.ETraceState`` 是一个与 ``brainstate.HiddenState`` 相对应的概念，它专门用于定义需要进行eligibility trace更新的模型状态。如果我们在程序中将模型的状态定义为 ``brainstate.HiddenState``，而非``brainscale.ETraceState``，那么``brainscale``的在线学习编译器将不再识别这个状态，从而其编译后的在线学习法则将不再流过该状态，导致模型的梯度更新出现错误或者遗漏。

### 3.2 ``ETraceParam``：模型参数

我们考虑一个简单的矩阵乘法算子，其计算公式为：

$$
y = W x + b
$$

其中，$W$是权重矩阵，$x$是输入向量，$b$是偏置向量。我们可以使用``ETraceParam``来定义这些模型参数。



In [15]:
def generate_weight(
    n_in, n_out, init: Callable = bst.init.KaimingNormal()
) -> brainscale.ETraceParam:
    weight = init([n_in, n_out])
    bias = bst.init.ZeroInit()([n_out])
    
    # 这里是最关键的一步，我们定义了一个 ETraceParam 类，用于描述权重矩阵和偏置向量
    return brainscale.ETraceParam({'weight': weight, 'bias': bias})

在上面的代码中，我们定义了一个``generate_weight``函数，它用于生成权重矩阵和偏置向量。这个函数返回一个``ETraceParam``对象，用于描述权重矩阵和偏置向量。

``brainscale.ETraceParam`` 是一个与 ``brainstate.ParamState`` 相对应的概念，它专门用于定义需要进行eligibility trace更新的模型参数。如果我们在程序中将模型的参数$\theta$定义为``brainscale.ETraceParam``，那么``brainscale``的在线学习编译器将会对这个参数进行具有时序依赖的梯度更新，即计算

$$\nabla_\theta \mathcal{L}=\sum_{t} \frac{\partial \mathcal{L}^{t}}{\partial \mathbf{h}^{t}} \sum_{k=1}^t \frac{\partial \mathbf{h}^t}{\partial \boldsymbol{\theta}^k} ,$$

其中，$\boldsymbol{\theta}^k$是第$k$时刻用到的权重$\boldsymbol{\theta}$。

相反，如果我们将模型的参数$\theta$定义为``brainstate.ParamState``，那么``brainscale``的在线学习编译器只会计算当前时刻损失函数对权重的偏导值，即

$$\nabla_\theta \mathcal{L}=\sum_{t} \frac{\partial \mathcal{L}^{t}}{\partial \mathbf{h}^{t}} \frac{\partial \mathbf{h}^t}{\partial \boldsymbol{\theta}^t}. $$

这意味着在``brainscale``的在线学习中，``brainstate.ParamState``将会被视为一个不需要进行eligibility trace更新的模型参数，失去对时序依赖信息的梯度计算。这样的设计可以更加灵活地控制模型参数的更新模式，增加模型参数梯度计算的可定制性。

### 3.3 ``ETraceOp``：模型输入输出函数

``ETraceOp``是另一个描述动力学交互的概念。``ETraceParam``描述了动力学交互中用到的参数，而``ETraceOp``描述了动力学交互的操作。它需要定义为一个具有如下格式要求的函数：

```python
def op(
    x: jax.Array, 
    param: brainscale.ETraceParam
) -> jax.Array:
    pass
```


``ETraceOp``描述了模型输入如何根据参数转换为输出，它可以实现为各种各样的模型的操作，包括线性矩阵乘法、稀疏矩阵乘法、卷积操作等。


针对上面的矩阵乘法算子，我们可以定义如下的``ETraceOp``：

In [16]:
@brainscale.ETraceOp
def matmul(x, w):
    weight = w['weight']
    bias = w['bias']
    return x @ weight + bias

### 3.4 ``ETraceParamOp`` = ``ETraceParam`` + ``ETraceOp``: 模型参数和操作的组合

可以看到，``ETraceParam`` 和 ``ETraceOp`` 总是相互耦合的。``ETraceParam`` 无法脱离函数转换进行独立参数化，而 ``ETraceOp`` 也无法脱离参数化进行独立输入输出转换操作。因此，我们可以将它们组合在一起，形成一个新的统一的概念：``ETraceParamOp``。``ETraceParamOp``是一个描述模型参数和操作的组合的概念，它包含了模型参数和操作的信息，因此需要接收一个参数和一个操作函数进行实例化。

因此，基于``ETraceParamOp``，上面的矩阵乘法算子可以被定义为：

In [17]:
class Linear(bst.nn.Module):
    """
    Linear layer.
    """
    def __init__(
        self,
        in_size: bst.typing.Size,
        out_size: bst.typing.Size,
        w_init: Callable = bst.init.KaimingNormal(),
    ):
        super().__init__()

        # input and output shape
        self.in_size = in_size
        self.out_size = out_size

        # weights
        weight = bst.init.param(w_init, [self.in_size[-1], self.out_size[-1]], allow_none=False)
        
        # operation
        op = lambda x, w: u.math.matmul(x, w)
        
        # 这里是最关键的一步，我们定义了一个 ETraceParamOp 类，用于描述权重矩阵和操作
        self.weight_op = brainscale.ETraceParamOp(weight, op)

    def update(self, x):
        # ETraceParamOp 的操作
        return self.weight_op.execute(x)

从上面的代码中可以看到，``ETraceParamOp``是一个描述模型参数和操作的组合的概念。它包含了模型参数和操作的信息，因此需要接收一个参数和一个操作函数进行实例化。``ETraceParamOp``的``execute``方法用于执行操作函数，将输入数据转换为输出数据。
至此，我们定义了一个简单的线性层模块``Linear``，它包含了一个权重矩阵和一个矩阵乘法操作。这个模块可以被用于构建支持在线学习的神经网络模型。

## 4. ``ETraceAlgorithm``：在线学习算法

``ETraceAlgorithm``是``brainscale``中的另一个重要概念，它定义了模型的状态更新过程中如何更新eligibility trace，以及定义了模型参数的梯度更新规则。``ETraceAlgorithm``是一个抽象类，专门用于描述``brainscale``内各种形式的在线学习算法。

``brainscale.ETraceAlgorithm``中提供的算法支持，是基于上面提供的``ETraceState``、``ETraceParam``和``ETraceOp``三个基本概念。``brainscale.ETraceAlgorithm``提供了一种灵活的在线学习编译器。它可以支持使用上述三个概念构建的任意神经网络模型的在线学习。


具体来说，目前 ``brainscale`` 支持的在线学习算法有：

- ``brainscale.DiagIODimAlgorithm``：该算法使用 ES-D-RTRL 算法进行在线学习，支持$O(N)$复杂度的在线梯度计算，适用于大规模脉冲神经网络模型的在线学习。具体算法细节可以参考[我们的论文](https://doi.org/10.1101/2024.09.24.614728)。
- ``brainscale.DiagParamDimAlgorithm``：该算法使用 D-RTRL 算法进行在线学习，支持$O(N^2)$复杂度的在线梯度计算，适用于循环神经网络模型和脉冲神经网络模型的在线学习。具体算法细节可以参考[我们的论文](https://doi.org/10.1101/2024.09.24.614728)。
- ``brainscale.DiagHybridDimAlgorithm``：该算法选择性地使用 ES-D-RTRL 或 D-RTRL 算法对模型参数进行在线学习。对于卷积层和高度稀疏连接的层，该算法有更大的倾向使用 D-RTRL 算法进行在线学习，以减少在线学习参数更新所需的计算复杂度。
- 未来，我们将会支持更多的在线学习算法，以满足更多的应用场景。




在下面的例子中，我们将展示如何使用``brainscale.ETraceAlgorithm``来构建一个支持在线学习的神经网络模型。


In [18]:
with bst.environ.context(dt=0.1 * u.ms):

    # 定义一个简单的LIF神经元构成的循环神经网络
    model = LIF_Delta_Net(10, 10)
    bst.nn.init_all_states(model)
    
    # 将该模型输入到在线学习算法中，以期进行在线学习
    model = brainscale.DiagIODimAlgorithm(model, decay_or_rank=0.99)
    
    # 根据一个输入数据编译模型的eligibility trace，
    # 此后，调用该模型不仅更新模型的状态，还会更新模型的eligibility trace
    example_input = bst.random.random(10) < 0.1
    model.compile_graph(example_input)


本质上，用户定义的神经网络模型只是规定了模型状态 $\mathbf{h}$ 如何随着输入和时间前向演化，而在线学习算法 ``ETraceAlgorithm`` 编译后则定义了模型eligibility trace $\mathbf{\epsilon}$ 如何随着模型状态的更新而更新。这样，当我们再次调用模型时，不仅会更新模型的状态，还会更新模型的eligibility trace。

In [19]:
with bst.environ.context(dt=0.1 * u.ms):
    
    out = model(example_input)

# 通过调用 model.etrace_xs 可以获取模型对突触前神经活动追踪的 eligibility trace 
bst.util.PrettyMapping(model.etrace_xs)

{
  Var(id=2319697265600):float32[20]: ShortTermState(
    value=Array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
           0., 0., 0.], dtype=float32)
  )
}

In [20]:
# 通过调用 model.etrace_dfs 可以获取模型对突触后神经活动追踪的 eligibility trace
bst.util.PrettyMapping(model.etrace_dfs)

{
  (Var(id=2319697265792):float32[10], Var(id=2319697273728):float32[10]): ShortTermState(
    value=Array([0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01],      dtype=float32)
  )
}

## 5. 总结

总的来说，`brainscale`为在线学习提供了一个完整而优雅的框架体系，其核心概念可以总结为以下几个层次：

1. **基础架构层**
   - 支持特定结构的动力学模型，将"动力学"和"交互"严格分离
   - 基于`brainstate`生态系统，完全兼容其编程范式
   - 通过`brainscale.nn`模块提供开箱即用的神经网络组件

2. **核心概念层**
   - `ETraceState`：标记需要进行eligibility trace更新的动力学状态
   - `ETraceParam`：标记需要进行eligibility trace更新的模型参数
   - `ETraceOp`：定义动力学交互的具体操作
   - `ETraceParamOp`：组合参数和操作的统一接口

3. **算法实现层**
   - `DiagIODimAlgorithm`：基于ES-D-RTRL算法，具有$O(N)$复杂度
   - `DiagParamDimAlgorithm`：基于D-RTRL算法，具有$O(N^2)$复杂度
   - `DiagHybridDimAlgorithm`：混合算法，根据网络结构特点自适应选择$O(N)$或$O(N^2)$复杂度的算法

4. **工作流程**
   - 使用基础组件构建神经网络模型
   - 选择合适的在线学习算法封装模型
   - 编译模型生成eligibility trace计算图
   - 通过前向传播同时更新模型状态和eligibility trace

这个框架的独特之处在于：它将复杂的在线学习算法封装在简洁的接口后，提供灵活的定制化机制，既保持高性能，又确保易用性。同时，它与现有的 `brainstate` 生态系统无缝集成。通过这样的设计，`brainscale` 使构建和训练在线学习神经网络变得直观而高效，为神经计算研究提供了强大的工具。
